In [19]:
# import numpy as np


# m=np.random.randint(30,40)
# n=np.random.randint(50,60)

# from scipy import linalg,sparse
# x=sparse.random(m,n,density=0.045,format='csr',dtype='float')*100
# mat=np.round(x.todense(),decimals=0)

# #remove void rows and columns
# rs=np.sum(mat,axis=1)
# cs=np.sum(mat,axis=0)
# rows=np.where(rs!=0)[0]
# rows=np.append(rows,m-1)
# cols=np.where(cs!=0)[1]
# mat=mat[:,cols][rows,:]
 
# rs=np.sum(mat,axis=1)
# cs=np.sum(mat,axis=0)

# #create supply demand
# demands= np.array(mat.sum(axis=1)).T[0]
# supplies=np.array(mat.sum(axis=0))[0]
# sparse=np.matrix(np.copy(mat), dtype = int)
# sparse[sparse>0]=1
# # sparse=np.array(sparse)

sparse=np.matrix([[0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1]])

supplies=np.reshape(np.array([92,32,93,83,78,138,55,75], dtype = int), 8)
demands=np.array([96,149,90,28,81,202])

demands, supplies

(array([ 96, 149,  90,  28,  81, 202]),
 array([ 92,  32,  93,  83,  78, 138,  55,  75]))

In [20]:
'''
disjoint - add current and exit
equal - update existing - no need to subtract existing subsets; no effect on existing subsets; update all supersets as well
proper subset - add current and update all supersets
proper supersets - add current with updated value using all existing disjoint subsets; Update all supersets as well, if exists
overlaps - add current and add new superset
'''

def update_constraints_list(current_LHS, current_RHS, existing_LHS, existing_RHS, supplies):
    
    new_LHS = existing_LHS.copy()
    new_RHS = existing_RHS.copy()
    
    bitwise_intersection = current_LHS & new_LHS
    
    all_disjoint_subsets = get_all_disjoint_subsets(current_LHS, new_LHS, bitwise_intersection)
    all_supersets = get_all_supersets(current_LHS, new_LHS, bitwise_intersection)
    all_other_overlaps = get_all_overlaps(current_LHS, new_LHS, bitwise_intersection)
    
    if (current_LHS == existing_LHS).any():
        ## constraint with same set already exists
        all_supersets = np.append(all_supersets, current_LHS)
        for current_constraint in all_supersets:
            new_RHS = modify_constraint(current_constraint, current_RHS, new_LHS, new_RHS)
        ## exit
        
    elif all_disjoint_subsets.shape[0] > 0 or all_supersets.shape[0] > 0 or all_other_overlaps.shape[0] > 0:
        ## constraint with same set doesn't exist but subsets/supersets exist
        ## Update the supersets first
        for current_constraint in all_supersets:
            new_RHS = modify_constraint(current_constraint, current_RHS, new_LHS, new_RHS)
        
        ## Update current constraint based on disjoint subsets and add it to constraints list
        current_supply = get_supply_for_combination(current_LHS, supplies)
        current_demand_including_subset = current_RHS
        for current_subset in all_disjoint_subsets:
            subset_demand = get_demand_for_existing_constraint(current_subset, new_LHS, new_RHS)
            current_demand_including_subset = current_demand_including_subset + subset_demand
        current_RHS_including_subset = current_supply - current_demand_including_subset
        new_LHS, new_RHS = add_constraint(current_LHS, current_RHS_including_subset, new_LHS, new_RHS)
        
        ## For oevrlap list keep creating new supersets and add them; skip if such superset already exist
        if all_other_overlaps.shape[0] > 0:
            for current_overlap in all_other_overlaps:
                new_superset_LHS = current_LHS | current_overlap
                if np.in1d(new_superset_LHS, all_supersets):
                    continue
                else:
                    overlap_demand = get_demand_for_existing_constraint(current_overlap, new_LHS, new_RHS)
                    new_superset_demand = overlap_demand + current_demand_including_subset #add current_B
                    new_superset_supply = get_supply_for_combination(new_superset_LHS, supplies)
                    new_superset_RHS = new_superset_supply - new_superset_demand #reduce from new_supply
                    new_LHS, new_RHS = add_constraint(new_superset_LHS, new_superset_RHS, new_LHS, new_RHS)
                    ## exit
    else:
        ## It is from different island
        ## add current constraint
        current_supply = get_supply_for_combination(current_LHS, supplies)
        current_RHS = current_supply - current_RHS
        new_LHS, new_RHS = add_constraint(current_LHS, current_RHS, new_LHS, new_RHS)
        ## exit
        
    new_LHS, new_RHS = remove_redundancy(new_LHS, new_RHS)
    
    return new_LHS, new_RHS

def get_all_disjoint_subsets(current_LHS, existing_LHS, bitwise_intersection):
    existing_LHS_subsets = existing_LHS[bitwise_intersection == existing_LHS]
    existing_LHS_subsets = np.setdiff1d(existing_LHS_subsets, current_LHS)
    existing_LHS_subsets = list(filter(lambda x: np.sum(x & existing_LHS_subsets == x) == 1, existing_LHS_subsets))
    return np.array(existing_LHS_subsets) 

def get_all_supersets(current_LHS, existing_LHS, bitwise_intersection):
    existing_LHS_supersets = existing_LHS[bitwise_intersection == current_LHS]
    existing_LHS_supersets = np.setdiff1d(existing_LHS_supersets, current_LHS)
    return existing_LHS_supersets

def get_all_overlaps(current_LHS, existing_LHS, bitwise_intersection):
    all_overlaps = existing_LHS[np.logical_and.reduce((bitwise_intersection != 0, bitwise_intersection != existing_LHS, bitwise_intersection != current_LHS))]
    removal_set = np.array([0])
    for element1 in all_overlaps:
        for element2 in all_overlaps:
            if element1 & element2 == element1 and element1 != element2 and (element1 | current_LHS[0] == element2 | current_LHS[0]): 
                #element1 is subset of element2; they are not equal and both have same superset
                removal_set = np.append(removal_set, element1)
    all_overlaps = np.setdiff1d(all_overlaps, np.unique(removal_set))        
    return all_overlaps

def add_constraint(current_LHS, current_RHS, existing_LHS, existing_RHS):
    new_LHS = np.append(existing_LHS, current_LHS)
    new_RHS = np.append(existing_RHS, current_RHS)
    return new_LHS, new_RHS

def modify_constraint(current_LHS, current_RHS, existing_LHS, existing_RHS):
    new_RHS = existing_RHS
    new_RHS[np.squeeze(np.array(existing_LHS == current_LHS))] -= current_RHS
    return new_RHS

def get_supply_for_combination(combination_LHS, supplies):
    import math
    bit_string = np.zeros(0, dtype = int)
    while combination_LHS>0:
        bit_string = np.append(bit_string, combination_LHS%2)
        combination_LHS = math.floor(combination_LHS/2)
    bit_string = np.flip(bit_string, 0)
    padding_length = supplies.shape[0] - bit_string.shape[0]
    bit_string = np.pad(bit_string, (padding_length,0), 'constant', constant_values=0)
    supply_for_combination = np.dot(bit_string, supplies)
    return supply_for_combination

def get_demand_for_existing_constraint(required_LHS, existing_LHS, existing_RHS):
    required_RHS = np.reshape(np.array(existing_RHS[existing_LHS == required_LHS]), 1)
    required_supply = get_supply_for_combination(required_LHS, supplies)
    required_demand = required_supply - required_RHS
    return required_demand

def remove_redundancy(all_LHS, all_RHS):
    new_LHS = all_LHS
    new_RHS = all_RHS
    for current_row in range(new_LHS.shape[0]):
        current_LHS = new_LHS[current_row]
        current_RHS = new_RHS[current_row]
        bitwise_intersection = current_LHS & new_LHS
        all_supersets = get_all_supersets(current_LHS, new_LHS, bitwise_intersection)
        for current_superset in all_supersets:
            superset_RHS = new_RHS[np.squeeze(np.array(new_LHS == current_superset))]
            if superset_RHS <= current_RHS:
                new_LHS[current_row] = 0
                new_RHS[current_row] = 0
                break
    new_RHS = np.delete(new_RHS, np.where(new_LHS == 0))
    new_LHS = np.delete(new_LHS, np.where(new_LHS == 0))
    return new_LHS, new_RHS

## Also redundancy should be checked on the sum of disjoint supersets

In [24]:
import datetime
start_time = datetime.datetime.now()

## initial constraints
column_count = sparse.shape[1]
A = np.concatenate([-1*np.identity(column_count),np.identity(column_count)])
b = np.concatenate([np.zeros(column_count), supplies])

#sparse = sparse[[0,3],:]

## Adding first constraint
request_LHS = np.apply_along_axis(lambda x: int(str(x).replace("[", "").replace("]", "").replace(" ", "").replace("\n", ""), 2), 1, sparse)
existing_LHS = np.reshape(request_LHS[0], 1)
existing_RHS = np.array([np.sum(np.array(sparse[0, :])*supplies) - demands[0]])

## Looping for other rows
for current_row in range(1, (sparse.shape[0] - 1)):
    current_LHS = np.reshape(request_LHS[current_row], 1)
    current_RHS = np.reshape(demands[current_row], 1)
    existing_LHS, existing_RHS = update_constraints_list(current_LHS, current_RHS, existing_LHS, existing_RHS, supplies)
    print(current_row, existing_LHS.shape, existing_RHS.shape)
    
end_time = datetime.datetime.now()


1 (1,) (1,)
2 (2,) (2,)
3 (3,) (3,)
4 (5,) (5,)


In [25]:
end_time - start_time

datetime.timedelta(0, 0, 6833)

In [22]:
existing_LHS

array([ 31,  63, 128, 224, 255])

In [23]:
existing_RHS

array([184, 187,  64, 108, 202])